# Mixed Panel Types - Plotly + Matplotlib

This notebook demonstrates **combining multiple visualization libraries** in a single Trelliscope display.

## Features Demonstrated

- ✅ **Mixed Panel Types** - Plotly interactive + matplotlib static in one display
- ✅ **Automatic Detection** - Viewer detects panel type from file extension
- ✅ **Unified Experience** - Seamless browsing across different panel types
- ✅ **Library Strengths** - Use each library where it excels
- ✅ **Format Flexibility** - PNG, HTML, SVG, JPEG all supported

## Use Case: Statistical Analysis Workflow

- **Matplotlib**: Static publication-quality statistical plots (Q-Q plots, histograms, heatmaps)
- **Plotly**: Interactive exploration plots (time series, scatter, box plots)
- **Combined**: Best of both worlds in one unified viewer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import sys
from scipy import stats

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent.parent))

from trelliscope import Display
from trelliscope.dash_viewer import create_dash_app

# Set matplotlib style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

In [ ]:
# Generate synthetic refinery margins data
from datetime import datetime, timedelta

countries = ['Algeria', 'Denmark', 'Germany', 'Japan', 'United Kingdom']
n_days = 180
start_date = datetime(2023, 1, 1)

data_rows = []
np.random.seed(42)

for country in countries:
    dates = [start_date + timedelta(days=i) for i in range(n_days)]
    
    # Generate realistic refinery and crude data with trends
    base_refinery = np.random.uniform(500, 2000)
    base_crude = np.random.uniform(400, 1800)
    
    trend_ref = np.linspace(0, np.random.uniform(-100, 100), n_days)
    trend_crude = np.linspace(0, np.random.uniform(-100, 100), n_days)
    
    seasonality_ref = 100 * np.sin(np.linspace(0, 4*np.pi, n_days))
    seasonality_crude = 80 * np.sin(np.linspace(0, 4*np.pi, n_days))
    
    noise_ref = np.random.randn(n_days) * 50
    noise_crude = np.random.randn(n_days) * 40
    
    refinery_kbd = base_refinery + trend_ref + seasonality_ref + noise_ref
    crude_kbd = base_crude + trend_crude + seasonality_crude + noise_crude
    
    for i in range(n_days):
        data_rows.append({
            'country': country,
            'date': dates[i],
            'refinery_kbd': max(0, refinery_kbd[i]),
            'crude_kbd': max(0, crude_kbd[i])
        })

df = pd.DataFrame(data_rows)

print(f"Generated {len(df):,} rows")
print(f"Countries: {len(df['country'].unique())}")

## Matplotlib: Publication-Quality Static Plots

Perfect for:
- Statistical diagnostics (Q-Q plots, residuals)
- Print publications
- Reports and papers
- Complex multi-panel figures

In [ ]:
def create_matplotlib_diagnostic(data, country):
    """
    Create 4-panel statistical diagnostic plot with matplotlib.
    
    Panels: histogram, Q-Q plot, time series, autocorrelation
    """
    fig = plt.figure(figsize=(10, 8))
    gs = gridspec.GridSpec(2, 2, hspace=0.3, wspace=0.3)
    
    values = data['refinery_kbd'].values
    
    # 1. Histogram with KDE
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.hist(values, bins=30, alpha=0.7, color='steelblue', edgecolor='black', density=True)
    
    # Add KDE
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(values)
    x_range = np.linspace(values.min(), values.max(), 100)
    ax1.plot(x_range, kde(x_range), 'r-', linewidth=2, label='KDE')
    
    ax1.set_xlabel('Refinery Capacity (kbd)')
    ax1.set_ylabel('Density')
    ax1.set_title('Distribution')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Q-Q Plot
    ax2 = fig.add_subplot(gs[0, 1])
    stats.probplot(values, dist="norm", plot=ax2)
    ax2.set_title('Q-Q Plot (Normality Check)')
    ax2.grid(True, alpha=0.3)
    
    # 3. Time Series with trend
    ax3 = fig.add_subplot(gs[1, 0])
    ax3.plot(data['date'], values, 'o-', alpha=0.6, linewidth=1, markersize=3, color='steelblue')
    
    # Add trend line
    x_numeric = np.arange(len(data))
    z = np.polyfit(x_numeric, values, 1)
    p = np.poly1d(z)
    ax3.plot(data['date'], p(x_numeric), 'r--', linewidth=2, label=f'Trend: {z[0]:.2f}x + {z[1]:.1f}')
    
    ax3.set_xlabel('Date')
    ax3.set_ylabel('Refinery (kbd)')
    ax3.set_title('Time Series with Trend')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.tick_params(axis='x', rotation=45)
    
    # 4. Autocorrelation
    ax4 = fig.add_subplot(gs[1, 1])
    from pandas.plotting import autocorrelation_plot
    autocorrelation_plot(pd.Series(values), ax=ax4, color='steelblue')
    ax4.set_title('Autocorrelation')
    ax4.set_xlabel('Lag')
    ax4.grid(True, alpha=0.3)
    
    # Main title
    fig.suptitle(f'{country} - Statistical Diagnostics', fontsize=14, fontweight='bold', y=0.995)
    
    return fig


def create_matplotlib_comparison(data, country):
    """
    Create comparison plot with correlation heatmap.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Scatter with regression
    ax1.scatter(data['refinery_kbd'], data['crude_kbd'], alpha=0.6, s=30, color='steelblue', edgecolors='black', linewidth=0.5)
    
    # Add regression line
    z = np.polyfit(data['refinery_kbd'], data['crude_kbd'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(data['refinery_kbd'].min(), data['refinery_kbd'].max(), 100)
    ax1.plot(x_line, p(x_line), 'r--', linewidth=2, label=f'y = {z[0]:.2f}x + {z[1]:.1f}')
    
    # Calculate correlation
    corr = data[['refinery_kbd', 'crude_kbd']].corr().iloc[0, 1]
    ax1.text(0.05, 0.95, f'r = {corr:.3f}', transform=ax1.transAxes, 
             fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    ax1.set_xlabel('Refinery Capacity (kbd)')
    ax1.set_ylabel('Crude Imports (kbd)')
    ax1.set_title('Refinery vs Crude Correlation')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Correlation heatmap
    corr_matrix = data[['refinery_kbd', 'crude_kbd']].corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
                square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax2,
                vmin=-1, vmax=1)
    ax2.set_title('Correlation Matrix')
    
    fig.suptitle(f'{country} - Correlation Analysis', fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    return fig


print("✓ Created matplotlib visualization functions")

## Plotly: Interactive Exploration Plots

Perfect for:
- Data exploration
- Interactive dashboards
- Presentations
- Drill-down analysis

In [ ]:
def create_plotly_interactive(data, country):
    """
    Create interactive Plotly chart with advanced features.
    """
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Interactive Time Series', 'Distribution Box Plots'),
        vertical_spacing=0.15,
        specs=[[{"secondary_y": True}], [{}]]
    )
    
    # Time series with dual y-axes
    fig.add_trace(
        go.Scatter(
            x=data['date'],
            y=data['refinery_kbd'],
            name='Refinery',
            line=dict(color='#2c7fb8', width=2),
            mode='lines+markers',
            marker=dict(size=4),
            hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Refinery: %{y:.1f} kbd<extra></extra>'
        ),
        row=1, col=1, secondary_y=False
    )
    
    fig.add_trace(
        go.Scatter(
            x=data['date'],
            y=data['crude_kbd'],
            name='Crude',
            line=dict(color='#d95f02', width=2, dash='dash'),
            mode='lines+markers',
            marker=dict(size=4),
            hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Crude: %{y:.1f} kbd<extra></extra>'
        ),
        row=1, col=1, secondary_y=True
    )
    
    # Box plots
    fig.add_trace(
        go.Box(y=data['refinery_kbd'], name='Refinery', marker_color='#2c7fb8', boxmean='sd'),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Box(y=data['crude_kbd'], name='Crude', marker_color='#d95f02', boxmean='sd'),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title=f"{country} - Interactive Analysis",
        height=600,
        hovermode='x unified',
        showlegend=True,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    fig.update_xaxes(showgrid=True, gridcolor='#e0e0e0')
    fig.update_yaxes(showgrid=True, gridcolor='#e0e0e0')
    
    return fig


print("✓ Created plotly visualization functions")

## Generate Mixed Panel Types

Each country gets:
1. Matplotlib diagnostic plot (4-panel static)
2. Matplotlib comparison plot (2-panel static)
3. Plotly interactive plot

In [ ]:
countries = sorted(df['country'].unique())

print(f"Generating mixed panels for {len(countries)} countries...\n")

mixed_data = []

for country in countries:
    country_df = df[df['country'] == country].copy()
    
    # Calculate statistics
    metrics = {
        'country': country,
        'avg_refinery': country_df['refinery_kbd'].mean(),
        'std_refinery': country_df['refinery_kbd'].std(),
        'correlation': country_df[['refinery_kbd', 'crude_kbd']].corr().iloc[0, 1],
        'n_obs': len(country_df)
    }
    
    # Matplotlib diagnostic (static PNG)
    mpl_diag = create_matplotlib_diagnostic(country_df, country)
    mixed_data.append({
        **metrics,
        'plot_type': 'Matplotlib Diagnostic',
        'format': 'PNG (static)',
        'panel': mpl_diag
    })
    plt.close(mpl_diag)
    
    # Matplotlib comparison (static PNG)
    mpl_comp = create_matplotlib_comparison(country_df, country)
    mixed_data.append({
        **metrics,
        'plot_type': 'Matplotlib Comparison',
        'format': 'PNG (static)',
        'panel': mpl_comp
    })
    plt.close(mpl_comp)
    
    # Plotly interactive (HTML)
    plotly_fig = create_plotly_interactive(country_df, country)
    mixed_data.append({
        **metrics,
        'plot_type': 'Plotly Interactive',
        'format': 'HTML (interactive)',
        'panel': plotly_fig
    })
    
    print(f"  ✓ {country}: 2 matplotlib + 1 plotly = 3 panels")

mixed_df = pd.DataFrame(mixed_data)
print(f"\n✓ Generated {len(mixed_df)} mixed-type panels")
print(f"  - Matplotlib (PNG): {len(mixed_df[mixed_df['format'] == 'PNG (static)'])}")
print(f"  - Plotly (HTML): {len(mixed_df[mixed_df['format'] == 'HTML (interactive)'])}")

In [ ]:
# Preview structure
print("DataFrame shape:", mixed_df.shape)
print("\nSample data:")
print(mixed_df[['country', 'plot_type', 'format', 'correlation']].head(9))

## Create Mixed Panel Display

The viewer automatically:
- Detects matplotlib figures → saves as PNG → displays as images
- Detects plotly figures → saves as HTML → extracts and renders natively
- Provides unified browsing experience across both types

In [ ]:
display = (
    Display(
        mixed_df,
        name="mixed_panel_types",
        description="Mixed Panel Types - Matplotlib (static) + Plotly (interactive)"
    )
    .set_panel_column("panel")
    .infer_metas()
    # Explicitly set plot_type and format as strings for multi-select dropdown filtering
    # replace=True is now the default, so these will overwrite inferred metas
    .add_meta_def("plot_type", "string")
    .add_meta_def("format", "string")
    .set_default_layout(ncol=3, nrow=2)
    .set_default_labels(["country", "plot_type", "format"])
)

print("✓ Display configured with mixed panel types")
print(f"  Total panels: {len(mixed_df)}")
print(f"  Countries: {len(countries)}")
print(f"  Plot types per country: 3 (2 matplotlib + 1 plotly)")

## Launch Viewer

### 🎯 What to Test

**Panel Type Detection:**
- 📊 Filter by `plot_type` to see only Matplotlib or Plotly
- 📄 Filter by `format` to see PNG vs HTML panels
- Notice: Plotly panels are **interactive** (hover, zoom, pan)
- Notice: Matplotlib panels are **static** (publication quality)

**Unified Experience:**
- Browse seamlessly across both panel types
- Click panels to view full-size (modal works for both types)
- Filter and sort work identically for all panel types

**Best of Both Worlds:**
- Use Plotly for interactive exploration
- Use Matplotlib for statistical diagnostics and publication plots
- All in one unified viewer!

The viewer opens at http://localhost:8055

In [ ]:
app = create_dash_app(display)
app.run(debug=False, host='127.0.0.1', port=8062)